# Dogs vs. Cats CNN

![Dogs vs. Cats CNN](dc.png)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split

def prepare_data(train_path, val_size=0.2, random_state=42):
    train_filenames = os.listdir(train_path)
    train_categories = ['dog' if filename.split(".")[0] == 'dog' else 'cat' for filename in train_filenames]

    df = pd.DataFrame({
        'filename': train_filenames,
        'category': train_categories
    })

    train_df, val_df = train_test_split(df, test_size=val_size, stratify=df["category"], random_state=random_state)
    return train_df, val_df

In [4]:
train_path = "train"
train_df, val_df = prepare_data(train_path)
print(f"Total Training Images: {len(train_df)}")
print(f"Total Validation Images: {len(val_df)}")

Total Training Images: 20000
Total Validation Images: 5000


In [5]:
train_df.head()

,filename,category
20022,dog.5518.jpg,dog
4993,cat.3241.jpg,cat
24760,dog.9783.jpg,dog
13775,dog.11144.jpg,dog
20504,dog.5952.jpg,dog


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_path,
    x_col='filename',
    y_col='category',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_path,
    x_col='filename',
    y_col='category',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    subset='validation'
)


Found 17000 validated image filenames belonging to 2 classes.
Found 3000 validated image filenames belonging to 2 classes.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout

IMAGE_SIZE = (128, 128, 3)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=IMAGE_SIZE),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,889 (429.25 KB)

 Trainable params: 109,889 (429.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    verbose=1
)

Epoch 1/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 422s 787ms/step - accuracy: 0.5556 - loss: 0.6779 - val_accuracy: 0.5633 - val_loss: 0.6731
Epoch 2/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 354s 666ms/step - accuracy: 0.6460 - loss: 0.6345 - val_accuracy: 0.6860 - val_loss: 0.5878
Epoch 3/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 210s 395ms/step - accuracy: 0.6934 - loss: 0.5884 - val_accuracy: 0.7377 - val_loss: 0.5500
Epoch 4/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 202s 379ms/step - accuracy: 0.7134 - loss: 0.5621 - val_accuracy: 0.6733 - val_loss: 0.6043
Epoch 5/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 205s 385ms/step - accuracy: 0.7402 - loss: 0.5278 - val_accuracy: 0.7610 - val_loss: 0.5008


In [14]:
class_names = ['cat', 'dog']   # veya ['dog', 'cat'] — modele göre

In [15]:
# --- Model kaydetme hücresi (Keras 3 uyumlu) ---
import json
import os
from tensorflow import keras

# Sınıf isimlerini tanımla (manuel veya otomatik)
class_names = ['cat', 'dog']   # veya list(train_generator.class_indices.keys())

save_dir = "saved_model_v1"
os.makedirs(save_dir, exist_ok=True)

# 1️⃣ Keras formatında kaydet
keras_path = os.path.join(save_dir, "model.keras")
model.save(keras_path)
print(f"Keras formatında kaydedildi: {keras_path}")

# 2️⃣ HDF5 formatında kaydet
h5_path = os.path.join(save_dir, "model.h5")
model.save(h5_path)
print(f"HDF5 formatında kaydedildi: {h5_path}")

# 3️⃣ Sınıf isimlerini JSON olarak kaydet
labels_path = os.path.join(save_dir, "class_names.json")
with open(labels_path, "w", encoding="utf-8") as f:
    json.dump(class_names, f, ensure_ascii=False)

print(f"Etiket dosyası kaydedildi: {labels_path}")

Keras formatında kaydedildi: saved_model_v1\model.keras
HDF5 formatında kaydedildi: saved_model_v1\model.h5
Etiket dosyası kaydedildi: saved_model_v1\class_names.json


In [16]:
from tensorflow.keras.utils import load_img, img_to_array

test_dir = "test1"

def load_test_images(test_dir, image_size=(128, 128)):
    image_ids = []
    images = []
    for filename in sorted(os.listdir(test_dir)):
        if filename.endswith(".jpg"):
            img_path = os.path.join(test_dir, filename)
            img = load_img(img_path, target_size=image_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
            image_ids.append(int(filename.split(".")[0]))
    return np.array(images), image_ids

test_images, test_ids = load_test_images(test_dir)
predictions = model.predict(test_images, batch_size=32, verbose=1)
predictions = (predictions > 0.5).astype(int).flatten()

submission_df = pd.DataFrame({"id": test_ids, "label": predictions})
submission_df = submission_df.sort_values(by="id")
submission_df.to_csv("submission.csv", index=False)

391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 95ms/step


### Summary

### Sonuç

Bu proje, derin öğrenmenin görüntü sınıflandırmasında başarılı bir şekilde uygulandığını göstermektedir.
İyi tasarlanmış bir CNN mimarisini ön işleme ve model optimizasyonuyla birleştirerek, köpekleri ve kedileri ayırt etmede yüksek doğruluk elde ediyoruz.
Streamlit uygulaması, kullanıcı dostu bir arayüz sunarak bunu pratik ve dağıtılabilir bir yapay zeka çözümüne dönüştürüyor.